In [175]:
# Necessary impports and pip installs
!pip install googlesearch-python
import pandas as pd
import os
import numpy as np
import re
from googlesearch import search
import requests
from bs4 import BeautifulSoup

python(18588) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [139]:
'''
Dataframe loading. 
drone_df : raw FAA drone intrusion info.
airport_codes_2 : huge dataset with info on airport codes, latitude & longitude
'''

path_df = "fy24_q2.xlsx"
drone_df = pd.read_excel(path_df)

path_3 = "all-airport-data.xlsx"
airport_codes_2 = pd.read_excel(path_3)

/Users/vishalivallioor/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [140]:
''''
Line of code to extract all the 3 letter airport codes. 
'''

list_codes = list(airport_codes_2["Loc Id"])

drone_df['code'] = None

for index_2, row in drone_df.iterrows():
  x = row["Summary"]
  splitted_x = x.split()
  punc = '''!()-[]{};:'"\,<>.,/?@#$%^&*_~'''

  # Removing punctuations in string
  # Using loop + punctuation string
  index = 0
  for word in splitted_x:
      for letter in word:
        if letter in punc:
          word = word.replace(letter, "")
          splitted_x[index] = word
      index = index + 1
  filtered_codes = list(set([i for i in splitted_x if len(i) == 3 and i not in ["FAA", "UAS", "OPS", "THE"]]))
  for i in filtered_codes:
    if i in list_codes:
       drone_df.at[index_2, 'code'] = i

drone_df

,Date,State,City,Summary,code
0,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",MIA
1,2024-01-01,TEXAS,FORT WORTH,"PRELIM INFO FROM FAA OPS: FORT WORTH, TX/UAS S...",FTW
2,2024-01-01,THE BAHAMAS,BAHAMAS,"PRELIM INFO FROM FAA OPS: BIMINI, BAHAMAS/UAS ...",FLL
3,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",HPN
4,2024-01-01,COLORADO,DENVER,"PRELIM INFO FROM FAA OPS: DENVER, CO/UAS SIGHT...",DEN
...,...,...,...,...,...
321,2024-03-15,PR,MAYAGUEZ,"PRELIM INFO FROM FAA OPS: MAYAGUEZ, PUERTO RIC...",OFF
322,2024-03-31,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",MYR
323,2024-03-31,CALIFORNIA,FRESNO,"PRELIM INFO FROM FAA OPS: FRESNO, CA/UAS SIGHT...",VIS
324,2024-03-31,NEW JERSEY,CALDWELL,"PRELIM INFO FROM FAA OPS: CALDWELL, NJ/UAS SIG...",CDW


In [230]:
'''
All regex commands for drone information extraction.
'''

def extract_direction(summary):
    match = re.search(r'WHILE\s([A-Z]+\sBOUND)', summary)
    if match:
        return match.group(1)
    else:
        return np.nan

drone_df['Direction'] = drone_df['Summary'].apply(extract_direction)

pattern_time = r'(?<=\/)([0-9]{4}[A-Z])'
drone_df['Time'] = drone_df['Summary'].str.extract(pattern_time)

pattern_advised = r'(?<=\/)([A-Z ]+)(?= ADVISED)'
drone_df['Authority Advised'] = drone_df['Summary'].str.extract(pattern_advised)

pattern_route = r'(?<=, )([A-Z  ,()]+ - [A-Z  ,()]+)(?=,)'
drone_df['Route'] = drone_df['Summary'].str.extract(pattern_route)

pattern_location = r'(?<= )([0-9]+ [(W|ESE|E|N|NW|SW|S|SE|NE|NNE|NNW|WNW|WSW|NORTHWEST|ENE|SSE|SSW)]+ [A-Z() ]+)(?= AT| FEET|[.])'
drone_df['Location'] = drone_df['Summary'].str.extract(pattern_location)

pattern_altitude = r'(?<= )(AT [0-9,]+ FEET)(?= )'
drone_df['Altitude'] = drone_df['Summary'].str.extract(pattern_altitude)

drone_df

,Date,State,City,Summary,code,Direction,Time,Authority Advised,Route,Location,Altitude,latitude,longitude
0,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",MIA,W BOUND,0001E,MIAMI ATCT,"PANAMA CITY, PANAMA (MPTO) - MIA",1 E MIA,AT 500 FEET,None,None
1,2024-01-01,TEXAS,FORT WORTH,"PRELIM INFO FROM FAA OPS: FORT WORTH, TX/UAS S...",FTW,ESE BOUND,0015C,MEACHAM ATCT,FTW - FTW,8 ESE FTW,"AT 1,900 FEET",None,None
2,2024-01-01,THE BAHAMAS,BAHAMAS,"PRELIM INFO FROM FAA OPS: BIMINI, BAHAMAS/UAS ...",FLL,E BOUND,0655E,MIAMI ARTCC,"FLL - GOVERNORS HARBOUR, BAHAMAS (MYEN)",7 E BIMINI VORTAC,"AT 9,000 FEET",None,None
3,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",HPN,NaN,1010E,MIA,MIA - HPN,4 W MIA,NaN,None,None
4,2024-01-01,COLORADO,DENVER,"PRELIM INFO FROM FAA OPS: DENVER, CO/UAS SIGHT...",DEN,N BOUND,0945M,DEN ATCT,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,2024-03-15,PR,MAYAGUEZ,"PRELIM INFO FROM FAA OPS: MAYAGUEZ, PUERTO RIC...",OFF,E BOUND,1143L,SAN JUAN CERAP,"LA ROMANA, DOMINICAN REPUBLIC (MDLR) - SJU",10 S MAYAGUEZ (TJMZ),"AT 9,000 FEET",None,None
322,2024-03-31,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",MYR,N BOUND,1210E,ATLANTA TRACON,MYR - ATL,20 SE ATL,"AT 5,000 FEET",None,None
323,2024-03-31,CALIFORNIA,FRESNO,"PRELIM INFO FROM FAA OPS: FRESNO, CA/UAS SIGHT...",VIS,SE BOUND,0920P,FRESNO ATCT,CCR - VIS,10 WNW FAT,"AT 8,000 FEET",None,None
324,2024-03-31,NEW JERSEY,CALDWELL,"PRELIM INFO FROM FAA OPS: CALDWELL, NJ/UAS SIG...",CDW,NaN,1156E,MCGUIRE APCH,CDW - UNKN,NaN,NaN,None,None


In [233]:
'''
Using Beautiful Soup and Google Search with Airnav in order to map the values for which long/lat could not be found.

'''
edge_df = drone_df[drone_df["code"].isna()]
edge_df_location = edge_df[~edge_df["Location"].isna()]

# Add new columns for latitude and longitude
drone_df['latitude'] = None
drone_df['longitude'] = None

# Iterate over rows with NaN "code" but valid "Location"
for idx, location in edge_df_location.iterrows():
    loc_list = location['Location'].split()[2:]
    full_string = " ".join(loc_list)
    query_plain = " airnav site:airnav.com"
    query = full_string + query_plain

    # Perform Google search and get the top result
    search_results = search(query, num_results=1)  
    final_url = next(search_results, None)
    
    if final_url:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
        }

        # Send request to the final_url
        response = requests.get(final_url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            faa_code = None
            latitude = None
            longitude = None
            
            # Find FAA code and Lat/Long
            table_rows = soup.find_all('tr')  
            for row in table_rows:
                # Extract FAA Identifier
                if 'FAA Identifier' in row.text:
                    faa_code = row.find_all('td')[1].text.strip()  
                
                # Extract Latitude and Longitude
                if 'Lat/Long' in row.text:
                    lat_long_text = row.find_all('td')[1].text.strip()
                    lat_long_split = lat_long_text.split("W")
                    
                    if len(lat_long_split) >= 2:
                        latitude = lat_long_split[0] + 'W' 
                        longitude = 'W' + lat_long_split[1]

            # Fallback regex for latitude and longitude extraction
            if not (latitude and longitude):
                string_soup = str(soup)
                result_re = re.findall(r'(?<=Lat\/Long:\s)([0-9\-\.]+[N|S])\s\/\s([0-9\-\.]+[E|W])', string_soup)
                if result_re:
                    latitude, longitude = result_re[0]

            # Update the "code" and new columns "latitude" and "longitude" in the drone_df DataFrame
            if faa_code:
                drone_df.at[idx, 'code'] = faa_code
            if latitude and longitude:
                drone_df.at[idx, 'latitude'] = latitude
                drone_df.at[idx, 'longitude'] = longitude
        else:
            print(f"Failed to fetch the page for {location['Location']}, status code: {response.status_code}")

In [235]:
drone_df[~drone_df["latitude"].isna()]

,Date,State,City,Summary,code,Direction,Time,Authority Advised,Route,Location,Altitude,latitude,longitude
23,2024-01-05,NEW JERSEY,ROBBINSVILLE,"PRELIM INFO FROM FAA OPS: ROBBINSVILLE, NJ/UAS...",None,SE BOUND,1522E,ATLANTIC CITY ATCT,NaN,5 S ROBBINSVILLE VORTAC (RBV),"AT 3,000 FEET",40-12-08.648N,074-29-42.094W
86,2024-02-04,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",None,NW BOUND,1853E,MIAMI ARTCC,NaN,19 N DHP,"AT 12,000 FEET",25-47-59.866N,080-20-56.526W
172,2024-02-23,CALIFORNIA,POMONA,"PRELIM INFO FROM FAA OPS: POMONA, CA/UAS SIGHT...",None,W BOUND,2052E,SOUTHERN CALIFORNIA TRACON,NaN,5 W POMONA VORTAC (POM),"AT 3,500 FEET",34-04-42.196N,117-47-13.463W
181,2024-02-24,TEXAS,GALVESTON,"PRELIM INFO FROM FAA OPS: GALVESTON, TX/UAS SI...",None,SE BOUND,1708C,HOUSTON TRACON,NaN,8 N SCHOLES VOR (VUH),"AT 3,000 FEET",29-16-09.604N,094-52-03.811W
268,2024-03-18,NEW YORK,DEER PARK,"PRELIM INFO FROM FAA OPS: DEER PARK, NY/UAS SI...",None,NW BOUND,1635E,REPUBLIC ATCT,NaN,5 SW DEER PARK VOR (DPK),AT 400 FEET,40-47-30.360N,073-18-13.239W
